In [11]:
!pip install selenium

In [15]:
import selenium

In [16]:
from selenium import webdriver

In [17]:
browser = webdriver.Chrome("chromedriver.exe")

In [18]:
url = "https://www.zomato.com/hyderabad/great-food-no-bull"
browser.get(url)

In [55]:
import time

while True:
    try:
        browser.find_element_by_css_selector("div.load-more").click()
        time.sleep(2)
    except:
        break

In [20]:
res_tags= browser.find_elements_by_css_selector("div.relative.top-res-box.entity-ads-snippet-track")
len(res_tags)

88

In [27]:
import pandas as pd

In [34]:
restaurent = pd.DataFrame()
for res in res_tags:
    title= res.find_element_by_css_selector("div.res_title").text
    loc = res.find_element_by_css_selector("div.nowrap.grey-text.fontsize5.ttupper").text
    food_type = res.find_elements_by_css_selector("div.nowrap.grey-text")
    food_type= food_type[1].text
    res_id = res.get_attribute("data-res-id")
    url= res.find_element_by_css_selector("a[data-link-type]")
    url=url.get_attribute("href")
    rating = res.get_attribute("data-rating")
    curr_res = {"title":title,"location":loc,"food_type":food_type,"res_id":res_id,"rating":rating,"url":url}
    restaurent=restaurent.append(curr_res,ignore_index=True,)
restaurent.head()

,food_type,location,rating,res_id,title,url
0,"Asian, Chinese, Continental, North Indian, Sou...","THE WESTIN HYDERABAD MINDSPACE, HITECH CITY, H...",4.5,91808,Seasonal Tastes - The Westin,https://www.zomato.com/hyderabad/seasonal-tast...
1,"Cafe, Pizza","JUBILEE HILLS, HYDERABAD",4.6,19113153,Zozoz Pizzeria,https://www.zomato.com/hyderabad/zozoz-pizzeri...
2,"Mediterranean, Italian, European, Salad","JUBILEE HILLS, HYDERABAD",4.5,93766,Olive Bistro,https://www.zomato.com/hyderabad/olive-bistro-...
3,"Healthy Food, Salad, Desserts, Cafe","JUBILEE HILLS, HYDERABAD",4.6,18563817,Santè Spa Cuisine,https://www.zomato.com/hyderabad/sant%C3%A8-sp...
4,"North Indian, Continental, Italian, Thai","HITECH CITY, HYDERABAD",4.5,19064101,TASALA,https://www.zomato.com/hyderabad/tasala-hitech...


In [53]:
url = restaurent.loc[4,"url"]
browser.get(url)

In [56]:
df_reviews= pd.DataFrame()
sel = "div.ui.segment.clearfix.brtop"
review_tags= browser.find_elements_by_css_selector(sel)
#res_id = restaurent.loc[0,"res_id"]
for reviews in review_tags:
    rev_name= reviews.find_element_by_css_selector("a[data-entity_id]").text
    rev_id = reviews.find_element_by_css_selector("a[data-entity_id]").get_attribute("data-entity_id")
    sel = "span.grey-text.fontsize5.nowrap"
    rev_count= reviews.find_element_by_css_selector(sel).text
    date = reviews.find_element_by_css_selector("time").get_attribute("datatime")
    rating_sel = "div.ttupper.fs12px.left.bold.zdhl2.tooltip"
    rating = reviews.find_element_by_css_selector(rating_sel).get_attribute("aria-label")
    text= reviews.find_element_by_css_selector("div.rev-text").text
    df_reviews=df_reviews.append({"rev_name":rev_name,
                                 "rev_id":rev_id,
                                 "rev_count":rev_count,
                                 "date":date,
                                 "rating":rating,
                                 "text":text},ignore_index=True)

In [58]:
df_reviews.shape

(48, 6)

In [62]:
def get_restaurent_reviews(browser,url):
    browser.get(url)
    while True:
        try:
            browser.find_element_by_css_selector("div.load-more").click()
            time.sleep(2)
        except:
            break
    df_reviews= pd.DataFrame()
    sel = "div.ui.segment.clearfix.brtop"
    review_tags= browser.find_elements_by_css_selector(sel)
    #res_id = restaurent.loc[0,"res_id"]
    for reviews in review_tags:
        rev_name= reviews.find_element_by_css_selector("a[data-entity_id]").text
        rev_id = reviews.find_element_by_css_selector("a[data-entity_id]").get_attribute("data-entity_id")
        sel = "span.grey-text.fontsize5.nowrap"
        rev_count= reviews.find_element_by_css_selector(sel).text
        date = reviews.find_element_by_css_selector("time").get_attribute("datatime")
        rating_sel = "div.ttupper.fs12px.left.bold.zdhl2.tooltip"
        rating = reviews.find_element_by_css_selector(rating_sel).get_attribute("aria-label")
        text= reviews.find_element_by_css_selector("div.rev-text").text
        df_reviews=df_reviews.append({"rev_name":rev_name,
                                     "rev_id":rev_id,
                                     "rev_count":rev_count,
                                     "date":date,
                                     "rating":rating,
                                     "text":text},ignore_index=True)
        return df_reviews

In [70]:
all_res_review= pd.DataFrame()
for index,res in restaurent.head(50).iterrows():
    try:
        res_reviews = get_restaurent_reviews(browser,res["url"])
        res_reviews["res_name"]=res["title"]
        res_reviews["res_id"]=res["res_id"]
        all_res_review= all_res_review.append(res_reviews,ignore_index=True)
        all_res_review.shape
    except:
        pass

In [75]:
all_res_review.head()

,date,rating,rev_count,rev_id,rev_name,text,res_name,res_id
0,None,Rated 5.0,"105 Reviews , 1210 Followers",38715577,Deepali Bohra,,Seasonal Tastes - The Westin,91808
1,None,Rated 3.0,"76 Reviews , 154 Followers",71442666,Food&Bikes (FaB),,Zozoz Pizzeria,19113153
2,None,Rated 4.5,"297 Reviews , 3567 Followers",33376100,_thefoodtrip_,RATED\n This is a beautiful place and one of ...,Olive Bistro,93766
3,None,Rated 3.0,"65 Reviews , 257 Followers",91052,Hardik Parekh,RATED\n Being one of the many high rated rest...,TASALA,19064101
4,None,Rated 5.0,"19 Reviews , 108 Followers",29627013,Katkar Jyothsna,RATED\n When you want to try something differ...,Piccolo Gelato,18616468
